# 影像處理作業：旋轉影像（最近鄰插值 / 雙線性插值 / OpenCV）
此作業實作三種旋轉影像的方式，分別為最近鄰插值、雙線性插值以及 OpenCV 提供的旋轉方法。

In [ ]:
import cv2
import numpy as np
import os

## 建立基礎影像
建立一個黑底白方塊的影像作為旋轉的基礎影像。

In [ ]:
def create_base_image(size=256, square_size=100):
    img = np.zeros((size, size), dtype=np.uint8)
    start = (size - square_size) // 2
    end = start + square_size
    img[start:end, start:end] = 255
    return img

## 最近鄰插值旋轉（手動）
使用最近鄰插值的方式手動實作影像旋轉。

In [ ]:
def rotate_nearest(img, angle_deg):
    angle_rad = - np.deg2rad(angle_deg)
    h, w = img.shape
    cx, cy = w // 2, h // 2
    rotated = np.zeros_like(img)

    for y in range(h):
        for x in range(w):
            tx = x - cx
            ty = y - cy
            src_x =  np.cos(angle_rad) * tx + np.sin(angle_rad) * ty + cx
            src_y = -np.sin(angle_rad) * tx + np.cos(angle_rad) * ty + cy
            src_x = int(round(src_x))
            src_y = int(round(src_y))
            if 0 <= src_x < w and 0 <= src_y < h:
                rotated[y, x] = img[src_y, src_x]
    return rotated

## 雙線性插值旋轉（手動）
使用雙線性插值的方式手動實作影像旋轉。

In [ ]:
def rotate_bilinear(img, angle_deg):
    angle_rad = - np.deg2rad(angle_deg)
    h, w = img.shape
    cx, cy = w // 2, h // 2
    rotated = np.zeros_like(img)

    for y in range(h):
        for x in range(w):
            tx = x - cx
            ty = y - cy
            src_x =  np.cos(angle_rad) * tx + np.sin(angle_rad) * ty + cx
            src_y = -np.sin(angle_rad) * tx + np.cos(angle_rad) * ty + cy

            x0 = int(np.floor(src_x))
            y0 = int(np.floor(src_y))
            x1 = x0 + 1
            y1 = y0 + 1

            if 0 <= x0 < w and 0 <= y0 < h and 0 <= x1 < w and 0 <= y1 < h:
                Ia = img[y0, x0]
                Ib = img[y0, x1]
                Ic = img[y1, x0]
                Id = img[y1, x1]

                wa = (x1 - src_x) * (y1 - src_y)
                wb = (src_x - x0) * (y1 - src_y)
                wc = (x1 - src_x) * (src_y - y0)
                wd = (src_x - x0) * (src_y - y0)

                value = wa * Ia + wb * Ib + wc * Ic + wd * Id
                rotated[y, x] = int(value)
    return rotated

## OpenCV旋轉（指定插值方式）
使用 OpenCV 提供的函數進行影像旋轉，並指定插值方式。

In [ ]:
def rotate_opencv(img, angle, method):
    h, w = img.shape
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
    return cv2.warpAffine(img, M, (w, h), flags=method, borderValue=0)

## Gradio 回呼函數
定義 Gradio 的回呼函數，將不同旋轉方式的結果返回。

In [ ]:
def update(angle):
    base = create_base_image()
    nn_manual = rotate_nearest(base, angle)
    nn_cv = rotate_opencv(base, angle, cv2.INTER_NEAREST)
    bilinear_manual = rotate_bilinear(base, angle)
    bilinear_cv = rotate_opencv(base, angle, cv2.INTER_LINEAR)

    return (
        Image.fromarray(base),
        Image.fromarray(nn_manual),
        Image.fromarray(nn_cv),
        Image.fromarray(bilinear_manual),
        Image.fromarray(bilinear_cv)
    )

## 設定 Gradio 介面
使用 Gradio 建立互動式介面，讓使用者可以調整旋轉角度並比較不同插值方式的結果。

In [ ]:
demo = gr.Interface(
    fn=update,
    inputs=gr.Slider(0, 360, value=30, step=1, label="Rotation Angle (degrees)"),
    outputs=[
        gr.Image(type="pil", label="Original Image"),
        gr.Image(type="pil", label="Nearest Neighbor (Manual)"),
        gr.Image(type="pil", label="Nearest Neighbor (OpenCV)"),
        gr.Image(type="pil", label="Bilinear (Manual)"),
        gr.Image(type="pil", label="Bilinear (OpenCV)")
    ],
    title="Image Rotation: Manual vs OpenCV Interpolation",
    description="Manually implemented nearest/bilinear interpolation vs OpenCV interpolation. Adjust rotation angle to compare."
)

## 啟動介面
執行以下程式碼以啟動 Gradio 介面。

In [ ]:
demo.launch()